<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_4"><div id="image_img" class="header_image_4"></div></td>
        <td class="header_text"> Generation of Poincar&eacute; Plot from ECG Analysis</td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">pre-process&#9729;ecg&#9729;poincare</td> 
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

The <strong><span class="color2">Poincar&#x00E9; Plot</span></strong> defines a geometrical and non-linear approach for studying heart rate variability and in a more generic perspective is also a methodology to quantify the level of chaos in periodic processes.

This structure consists in a bidimensional plot, where each of its points has as coordinates the duration of two consecutive RR intervals (<strong><span class="color7">$RR[i]$, $RR[i + 1]$</span></strong>), i.e. from tachogram each pair of samples <strong><span class="color13">i</span></strong> and <strong><span class="color13">i + 1</span></strong> define a point in the Poincar&#x00E9; Plot.

There are two reference lines mathematically defined by the following equations:
<table width="100%">
    <tr>
        <td width="50%">
            \begin{equation}
                RR[i + 1] = RR[i]
            \end{equation}
        </td>
        <td width="50%">
            \begin{equation}
                RR[i + 1] = -RR[i] + 2 avg(RR)
            \end{equation}
        </td>
    </tr>
</table>

If the dispersion of Poincar&#x00E9; points in relation to these lines is high, than it can be concluded that heart rate variability is high.

<hr>

<p class="steps">1 - Importing needed packages</p>

In [1]:
# biosignalsnotebooks own package for loading and plotting the acquired data
import biosignalsnotebooks as bsnb

# Scientific packages
import numpy
import math

<p class="steps">2 - Load of acquired ECG data</p>

In [2]:
# Load of data (using a relative path of the project)
data, header = bsnb.load("../../signal_samples/ecg_5_min.h5", get_header=True)

<p class="steps">3 - Identification of mac address of the device and the channel used during acquisition</p>

In [3]:
channel = list(data.keys())[0]

In [4]:
print ("Mac Address: " + str(header["device name"]) + " Channel: " + str(channel))

Mac Address: b'00:07:80:58:9B:3F' Channel: CH1


<p class="steps">4 - Storage of sampling frequency and acquired data inside variables</p>

In [5]:
# Sampling frequency and acquired data
fs = header["sampling rate"]

# Signal Samples
signal = data[channel]
time = numpy.linspace(0, len(signal) / fs, len(signal))

<p class="steps">5 - Generation of tachogram</p>

In [6]:
tachogram_data, tachogram_time = bsnb.tachogram(signal, fs, signal=True, out_seconds=True)

<p class="steps">6 - Axes of Poincar&#x00E9; plot</p>

In [7]:
rr_i = tachogram_data[:-1]
rr_i_plus_1 = tachogram_data[1:]

<p class="steps">7 - Generation of Poincar&#x00E9; Plot</p> 

In [8]:
bsnb.dispersion(rr_i, rr_i_plus_1, 'RR\u1D62 (s)', 'RR\u1D62\u208A\u2081 (s)')

[Figure(id='1001', ...)]

<p class="steps">8 - Determination of SD1 and SD2 parameters</p>
These parameters quantify the dispersion level of the Poincar&#x00E9; samples regarding to each reference line, being proportional to the short and long-term heart rate variability, respectively.
<table width="100%">
    <tr>
        <td width="50%">
            \begin{equation}
                SD1 = \sqrt{\frac{SDSD^2}{2}}
            \end{equation}
        </td>
        <td width="50%">
            \begin{equation}
                SD2 = \sqrt{2 \times SDNN^2 - SD1^2}
            \end{equation}
        </td>
    </tr>
</table>
where:
<table width="100%">
    <tr>
        <td width="50%">
            \begin{equation}
                SDSD = \sqrt{\frac{\sum_{i=2}^N ((RR[i] - RR[i - 1]) - \overline{RR_i - RR_{i - 1}})^2}{N - 1}}
            \end{equation}
        </td>
        <td width="50%">
            \begin{equation}
                SDNN = \sqrt{\frac{\sum_{i=1}^N (RR[i] - \overline{RR})^2}{N - 1}}
            \end{equation}
        </td>
    </tr>
</table>
and:
<table style="text-align:left">
    <tr>
        <td>
            $\overline{RR_i - RR_{i - 1}}$ is the average value of the first derivative of tachogram, while 
            $\overline{RR}$ is the average value of the tachogram samples
        </td>
    </tr>
</table>

In [9]:
tachogram_diff = numpy.diff(tachogram_data)
SDSD = numpy.std(tachogram_diff)
SDNN = numpy.std(tachogram_data)
        
SD1 = numpy.sqrt(0.5 * numpy.power(SDSD, 2))
SD2 = numpy.sqrt(2 * numpy.power(SDNN, 2) - numpy.power(SD1, 2))

avg_rr = numpy.average(tachogram_data)

<p class="steps">9 - Representation of Poincar&#x00E9; Plot with associated parameters</p>

In [10]:
list_figures = bsnb.plot_poincare(tachogram_data)

*This procedure can be automatically done by **poincare** function in **process** module of **<span class="color2">biosignalsnotebooks</span>** package*

In [11]:
x_axis, y_axis, SD1, SD2 = bsnb.poincare(signal, fs, signal=True, in_seconds=False)

<strong><span class="color7">We hope that you have enjoyed this guide. </span><span class="color2">biosignalsnotebooks</span><span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining <a href="../MainFiles/biosignalsnotebooks.ipynb">Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></span></strong> ! 

<span class="color6">**Auxiliary Code Segment (should not be replicated by the user)**</span>

In [12]:
bsnb.css_style_apply()

.................... CSS Style Applied to Jupyter Notebook .........................


In [13]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'],
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>